In [11]:
import time
import math
import builtins  # para garantir o uso do max() nativo do Python
from pyspark.sql.functions import col
from pyspark import StorageLevel
from delta.tables import DeltaTable

# ============================================================
# 0. Configurações de Segurança e Performance
# ============================================================

# Desliga a otimização automática de escrita do Fabric para respeitar
# nosso particionamento manual (Round Robin).
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")

# Habilita Adaptive Query Execution (bom para joins, aggs e skew em geral)
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

# Caminho da tabela de destino
path_destino = (
    "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/"
    "lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_ultima_posicao_mes"
)

# Quantidade alvo de linhas por arquivo (ajuste conforme sua realidade)
LINHAS_POR_ARQUIVO = 200_000

# Horas de retenção para o VACUUM (time travel mínimo preservado)
RETENCAO_VACUUM_HORAS = 1

# ============================================================
# 1. Leitura e Materialização (PERSIST)
#    Isola problemas de leitura e evita reprocessar tudo na escrita.
# ============================================================

print("1. Iniciando leitura e carregamento para memória (Persist)...")
start_read = time.time()

# Leitura da tabela histórica
df_historico = spark.table("tab_gold_dim_funcionario_historico")

# Filtro para pegar apenas a última posição do mês (RK_MES = 1)
df_filtrado = df_historico.filter(col("RK_MES") == 1)

# Persist em memória + disco (mais seguro que cache() se o dataset for grande)
df_cached = df_filtrado.persist(StorageLevel.MEMORY_AND_DISK)

# Força a execução da leitura e materialização
total_linhas = df_cached.count()

end_read = time.time()
print(f"   Leitura concluída em {end_read - start_read:.2f} segundos.")
print(f"   Total de linhas para gravar: {total_linhas}")

# ============================================================
# 2. Escrita com Round Robin (A Solução do Skew na saída)
# ============================================================

if total_linhas > 0:
    # Cálculo dinâmico da quantidade de arquivos finais
    num_arquivos_finais = builtins.max(1, math.ceil(total_linhas / LINHAS_POR_ARQUIVO))

    print(
        f"2. Iniciando gravação Round Robin em {num_arquivos_finais} arquivos "
        f"(~{LINHAS_POR_ARQUIVO} linhas por arquivo, se possível)..."
    )

    (
        df_cached
            # REPARTITION SEM COLUNAS = ROUND ROBIN
            # O Spark distribui as linhas aleatoriamente,
            # deixando os arquivos finais bem equilibrados.
            .repartition(num_arquivos_finais)
            .write
            .format("delta")
            .mode("overwrite")
            .option("overwriteSchema", "true")
            .save(path_destino)
    )

    print("   Gravação concluída com sucesso.")
else:
    print("   Nenhum dado encontrado para gravar. Escrita não executada.")

# Libera a memória/disk do cluster
df_cached.unpersist()

# ============================================================
# 3. Manutenção Pós-Gravação (Limpeza com VACUUM)
#    Remove arquivos antigos não referenciados, evitando lixo físico.
# ============================================================

print("3. Executando limpeza física (VACUUM)...")

# Desabilita temporariamente a trava de segurança de 7 dias
# (use com cuidado; aqui estamos confiando que não há necessidade
# de manter versões muito antigas dessa tabela).
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

delta_table = DeltaTable.forPath(spark, path_destino)

# Apaga fisicamente qualquer arquivo que não faça parte das versões
# dentro da janela de retenção (ex.: 1 hora).
delta_table.vacuum(RETENCAO_VACUUM_HORAS)

# Reabilita a trava de segurança
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "true")

print(
    f"Processo finalizado. Tabela otimizada, com arquivos equilibrados e "
    f"lixo físico limpo (retenção = {RETENCAO_VACUUM_HORAS} hora(s))."
)

StatementMeta(, 500887ec-2bd9-45ad-b4b3-7218fd3fb173, 13, Finished, Available, Finished)

1. Iniciando leitura e carregamento para memória (Persist)...
   Leitura concluída em 0.71 segundos.
   Total de linhas para gravar: 827268
2. Iniciando gravação Round Robin em 5 arquivos (~200000 linhas por arquivo, se possível)...
   Gravação concluída com sucesso.
3. Executando limpeza física (VACUUM)...
Processo finalizado. Tabela otimizada, com arquivos equilibrados e lixo físico limpo (retenção = 1 hora(s)).
